In [10]:
import pandas as pd
import numpy as np
from typing import Optional

class DataPrepKit:
    """
    A helper class for preprocessing employee datasets using Pandas and NumPy.
    This class provides methods to read data, summarize it, handle missing values,
    encode categorical variables, and save processed data.
    """

    def __init__(self):
        """
        Initializes the DataPrepKit class by allowing the user to upload a file.
        """
        file_path = "HR_Analytics.csv"
        self.data = self.read_data(file_path)

    def read_data(self, file_path: str) -> pd.DataFrame:
        """
        Reads an employee dataset from a given file path and returns a Pandas DataFrame.

        :param file_path: Path to the dataset file.
        :return: Loaded Pandas DataFrame.
        """
        try:
            if file_path.endswith(".csv"):
                return pd.read_csv(file_path)
            elif file_path.endswith(".xlsx") or file_path.endswith(".xls"):
                return pd.read_excel(file_path)
            elif file_path.endswith(".json"):
                return pd.read_json(file_path)
            else:
                raise ValueError("Unsupported file format. Please upload a CSV, Excel, or JSON file.")
        except Exception as e:
            print(f"Error loading file: {e}")
            return pd.DataFrame()

    def summarize_data(self) -> pd.DataFrame:
        """
        Generates a summary of the employee dataset, including mean, median, mode, and missing values.

        :return: Pandas DataFrame with summary statistics.
        """
        summary = self.data.describe(include='all').transpose()
        summary['missing_values'] = self.data.isnull().sum()
        return summary

    def handle_missing_values(self) -> None:
        """
        Handles missing values by allowing user to choose a strategy.
        """
        strategy = input("Choose strategy for handling missing values (drop/mean/median/mode): ")
        column = input("Enter column name to apply strategy (leave blank for all columns): ").strip()
        column = column if column else None

        if strategy == "drop":
            self.data.dropna(inplace=True)
        elif strategy in ["mean", "median", "mode"]:
            if column:
                if strategy == "mean":
                    self.data[column].fillna(self.data[column].mean(), inplace=True)
                elif strategy == "median":
                    self.data[column].fillna(self.data[column].median(), inplace=True)
                elif strategy == "mode":
                    self.data[column].fillna(self.data[column].mode()[0], inplace=True)
            else:
                for col in self.data.select_dtypes(include=[np.number]):
                    if strategy == "mean":
                        self.data[col].fillna(self.data[col].mean(), inplace=True)
                    elif strategy == "median":
                        self.data[col].fillna(self.data[col].median(), inplace=True)
                    elif strategy == "mode":
                        self.data[col].fillna(self.data[col].mode()[0], inplace=True)
        else:
            print("Invalid strategy. Please choose 'drop', 'mean', 'median', or 'mode'.")

    def encode_categorical(self) -> None:
        """
        Encodes categorical features by allowing user to choose an encoding method.
        """
        method = input("Choose encoding method (onehot/label): ")
        categorical_cols = self.data.select_dtypes(include=['object']).columns

        if method == "onehot":
            self.data = pd.get_dummies(self.data, columns=categorical_cols)
        elif method == "label":
            from sklearn.preprocessing import LabelEncoder
            le = LabelEncoder()
            for col in categorical_cols:
                self.data[col] = le.fit_transform(self.data[col])
        else:
            print("Invalid encoding method. Choose 'onehot' or 'label'.")

    def get_data(self) -> pd.DataFrame:
        """
        Returns the processed employee dataset.

        :return: Pandas DataFrame.
        """
        return self.data

    def save_data(self) -> None:
        """
        Saves the processed employee dataset as an Excel (.xlsx) file.
        """
        output_path = input("Enter the output file name (without extension, default is 'processed_data.xlsx'): ").strip()
        if not output_path:
            output_path = "processed_data.xlsx"
        else:
            output_path += ".xlsx"
        
        try:
            with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
                self.data.to_excel(writer, index=False, sheet_name="Processed Data")
            print(f"Data successfully saved to {output_path}")
        except Exception as e:
            print(f"Error saving file: {e}")

# Example usage
if __name__ == "__main__":
    # Initialize DataPrepKit with user-uploaded file
    data_prep = DataPrepKit()

    # Print dataset summary
    print("Employee Data Summary:\n", data_prep.summarize_data())

    # Handle missing values based on user choice
    data_prep.handle_missing_values()

    # Encode categorical features based on user choice
    data_prep.encode_categorical()

    # Get and print processed data
    print("Processed Employee Data:\n", data_prep.get_data().head())

    # Save processed data to Excel
    data_prep.save_data()


Employee Data Summary:
                            count unique                     top  freq  \
Age                       1470.0    NaN                     NaN   NaN   
Attrition                   1470      2                      No  1233   
BusinessTravel              1470      3           Travel_Rarely  1043   
DailyRate                 1470.0    NaN                     NaN   NaN   
Department                  1470      3  Research & Development   961   
DistanceFromHome          1470.0    NaN                     NaN   NaN   
Education                 1470.0    NaN                     NaN   NaN   
EducationField              1470      6           Life Sciences   606   
EmployeeCount             1470.0    NaN                     NaN   NaN   
EmployeeNumber            1470.0    NaN                     NaN   NaN   
EnvironmentSatisfaction   1470.0    NaN                     NaN   NaN   
Gender                      1470      2                    Male   882   
HourlyRate                1

Choose strategy for handling missing values (drop/mean/median/mode):  mean
Enter column name to apply strategy (leave blank for all columns):  Age


C:\Users\hp\AppData\Local\Temp\ipykernel_14064\497810076.py:62: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.data[column].fillna(self.data[column].mean(), inplace=True)


Choose encoding method (onehot/label):  onehot


Processed Employee Data:
    Age  DailyRate  DistanceFromHome  Education  EmployeeCount  EmployeeNumber  \
0   41       1102                 1          2              1               1   
1   49        279                 8          1              1               2   
2   37       1373                 2          2              1               4   
3   33       1392                 3          4              1               5   
4   27        591                 2          1              1               7   

   EnvironmentSatisfaction  HourlyRate  JobInvolvement  JobLevel  ...  \
0                        2          94               3         2  ...   
1                        3          61               2         2  ...   
2                        4          92               2         1  ...   
3                        4          56               3         1  ...   
4                        1          40               3         1  ...   

   JobRole_Research Director  JobRole_Research S

Enter the output file name (without extension, default is 'processed_data.xlsx'):  processed_data.xlsx


Data successfully saved to processed_data.xlsx.xlsx


In [9]:
!pip install xlsxwriter
